In [1]:
import argparse
import pandas as pd

In [65]:
print('Make sure the pathway codes in the user provided file are formatted the same as it is throughout MetaGaia!!')
pathways_lst = []

Make sure the pathway codes in the user provided file are formatted the same as it is throughout MetaGaia!!


In [66]:
#Read in files
user_df = pd.read_csv('../../../../nitrogen_KOs2.tsv', sep='\t', index_col=False)
pathway_df = pd.read_csv('../../../output/final_merged_database.tsv', sep='\t', index_col=False)

#Verify database name is valid
databases_list = '[kegg, Cog, Pfam]'.upper().strip('[]').split(', ')
for check in databases_list:
    if check not in ['KEGG', 'COG', 'PFAM', 'EC_NUMBER']:
        print('Invalid database name was entered!')
        quit()
    else:
        #Convert to list
        user_df[check] = user_df[check].str.split(', ')
        index_cols = [col for col in user_df.columns if col != check]
        user_df = (user_df.set_index(index_cols)[check].apply(pd.Series).stack().reset_index().drop('level_'+str(len(index_cols)), axis=1).rename(columns={0:check}))
        for val in user_df[check]:
            if check == 'KEGG':
                if val[:3] != 'KO':
                    user_df.loc[user_df[check] == val, check] = 'KO:' + val
            elif check == 'PFAM':
                if val[:4] != 'pfam':
                    user_df.loc[user_df[check] == val, check] = 'pfam' + val[2:]
            elif check == 'EC_NUMBER':
                if val[:3] != 'EC':
                    user_df.loc[user_df[check] == val, check] = 'EC:' + val

In [57]:
#user_df = pd.read_csv('../../../../nitrogen_KOs2.tsv', sep='\t', index_col=False)
# databases_list = '[kegg, Cog, Pfam]'.upper().strip('[]').split(', ')
# user_df['COG'] = user_df['COG'].str.split(', ')
# user_df = (user_df.set_index([col for col in user_df.columns if col != 'COG'])['COG'].apply(pd.Series).stack().reset_index().rename(columns={0:'COG'}))
#user_df

#user_df.loc[user_df['KEGG'] == 'K20932']['KEGG']

#'K20932'[:3]


In [67]:
print('Beginning to extract pathway information.')
for d in databases_list:
    #Subset dataframe
    extracted_df = pathway_df[pathway_df[d].isin(user_df[d])]
    #If user provided file has multiple columns, map them to the extracted pathway dataframe
    remove_cols = [col for col in databases_list if col != d]
    extracted_df = extracted_df.drop(columns=remove_cols)
    if len(user_df.drop(columns=databases_list).columns.tolist()) > 1:
        extracted_df = extracted_df.merge(user_df.drop(columns=remove_cols), on=d, how='left')
    #Rename datbase column so that concatenation can occur
    cols_list = extracted_df.columns.tolist()
    cols_list[cols_list.index(d)] = 'Database'
    extracted_df.columns = cols_list
    #Add dataframes to list
    pathways_lst.append(extracted_df)

Beginning to extract pathway information.


In [68]:
#Combine all dataframes
final_df = pd.concat(pathways_lst)
final_df = final_df.set_index('Database')
#Save file
final_df.to_csv('../../../output/extracted_pathways.tsv', sep='\t', index=True)
print('Finished extracting information! File is saved in the \"output\" folder as \"extracted_pathways.tsv\".')

Finished extracting information! File is saved in the "output" folder as "extracted_pathways.tsv".
